In [13]:
#Data 
t_f = 25          #[year], lifetime of the ferries
t_b = 10

B_cost = 4000 #[SEK / kWh], cost of batteries for Eloise were approx 8 million. 8 million / 1260 kWH ~= 6350 kr / kWH

F_costHybrid = 77.5 * pow(10,6) - (1260 * 4000) #Cost of Eloise minus batteries
F_costRetrofit = 22.5 * pow(10,6) 
F_costDiesel = 62.5 * pow(10,6)


M_costDiesel = 800000
M_costHybrid = 700000    #[SEK / year], maintenance cost for hybrid/electric ferry
D_cost = 9           #[SEK / liter], diesel cost
D_cons = 35       #[liter / hour], diesel consumption
E_cost = 2             #[SEK / kWh], electricity cost
P_cons = 110       #[kWh / hour], electricity consumption

#Infrastructure:

P_cost = 3000 #SEK/kW, plug-in charger
W_cost = 4500 #SEK/kw, wirelss charger

#CRF
i = 0.03
n = t_f
CRF = (i*pow(1+i,n))/(pow(1+i,n)-1)
print(CRF)

#Depreciation formula
r = 0.073
N = 10

F_costHybridRetrofit = F_costDiesel*pow(1-r,N) + (F_costRetrofit - (1260 * B_cost)) #Depreciated value + retrofit cost excl. batteries
F_costNew130 = F_costHybrid + B_cost * 130 #New ferry with 130 kWh
F_costNew630 = F_costHybrid + B_cost * 630 #New ferry with 630 kWh


0.05742787103912777


In [14]:
#Acquisition cost (excl. batteries)
def aCalc(F_cost1,F_cost2,F_cost3,F_cost4): 
    C_a1 = (F_cost1 * CRF) / 365
    C_a2 = (F_cost2 * CRF) / 365
    C_a3 = (F_cost3 * CRF) / 365
    C_a4 = (F_cost4 * CRF) / 365
    return C_a1 + C_a2 + C_a3 + C_a4
#Battery part of acquisition cost       
def abCalc(kw1,kw2,kw3,kw4): 
    C_ab1 = (kw1 * B_cost * CRF) / 365
    C_ab2 = (kw2 * B_cost * CRF) / 365
    C_ab3 = (kw3 * B_cost * CRF) / 365
    C_ab4 = (kw4 * B_cost * CRF) / 365
    return C_ab1 + C_ab2 + C_ab3 + C_ab4
#Infrastructure cost:
def iCalc(P,W_s,W_f,P_char,W_char): #P = plug-in, W_s = stationary wireless, W_f = on-board wireless
    C_i = (((P * P_char * P_cost) + (W_s * W_char * W_cost * 0.6) + (W_f * W_char * W_cost * 0.4)) * CRF) / 365
    #print(C_i)
    return C_i
#Battery replacement cost:
def rCalc(kw1,kw2,kw3,kw4):
    C_r1 = (((t_f / t_b) * (B_cost * kw1)) * CRF) / 365
    C_r2 = (((t_f / t_b) * (B_cost * kw2)) * CRF) / 365
    C_r3 = (((t_f / t_b) * (B_cost * kw3)) * CRF) / 365
    C_r4 = (((t_f / t_b) * (B_cost * kw4)) * CRF) / 365
    return C_r1 + C_r2 + C_r3 + C_r4
#Maintenance cost:
def mCalc(M_cost1,M_cost2,M_cost3,M_cost4):
    C_m1 = M_cost1 / 365
    C_m2 = M_cost2 / 365
    C_m3 = M_cost3 / 365
    C_m4 = M_cost4 / 365
    return C_m1 + C_m2 + C_m3 + C_m4
#Fuel cost:
def fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4):
    C_f1 = (P_cons * E_cost * e_f1 * t_op1) + (D_cons * D_cost * (1-e_f1)*t_op1)
    C_f2 = (P_cons * E_cost * e_f2 * t_op2) + (D_cons * D_cost * (1-e_f2)*t_op2)
    C_f3 = (P_cons * E_cost * e_f3 * t_op3) + (D_cons * D_cost * (1-e_f3)*t_op3)
    C_f4 = (P_cons * E_cost * e_f4 * t_op4) + (D_cons * D_cost * (1-e_f4)*t_op4)
    return C_f1 + C_f2 + C_f3 + C_f4
#Total cost: 
def totCalc(C_a,C_ab,C_i,C_b,C_m,C_f):
    return C_a + C_ab + C_i + C_b + C_m + C_f
#Operational cost (defined as all cost except infrastructure and acquisition)
def opexCalc(C_tot,C_a,C_ab,C_i):
    return C_tot - C_i - C_a - C_ab

def summary(C_a,C_ab,C_i,C_b,C_m,C_f,C_tot,C_opex):
    print('COST SUMMARY [SEK / DAY]')
    print('')
    print('Acquisition cost: ' + str(C_a + C_ab))
    print('Acquisition cost (excl. batteries): ' + str(C_a))
    print('Battery part of acquisition cost: ' + str(C_ab))
    print('Infrastructure cost: ' + str(C_i))
    print('Battery replacement cost: ' + str(C_r))
    print('Maintenance cost: ' + str(C_m))
    print('Fuel cost: ' + str(C_f))
    print('Total scenario cost: ' +str(C_tot))
    print('Total Opex: ' +str(C_opex))

In [15]:
#BASELINE

t_op1 = 13       #[hour], the range is 10-13 hours. 13 is max since the ferries run from 06:00 to 19:00
t_op2 = 12.5
t_op3 = 8 
t_op4 = 0

e_f1 = 0.6154
e_f2 = 0.5200
e_f3 = 0
e_f4 = 0

f1_kw = 1260
f2_kw = 1000
f3_kw = 0
f4_kw = 0

P = 2
W_s = 0
W_f = 0
P_char = 200
W_char = 0

C_a = aCalc(F_costHybrid,F_costHybrid,F_costDiesel,F_costDiesel)
C_ab = abCalc(1260,1000,0,0)
C_i = iCalc(P,W_s,W_f,P_char,W_char)
C_r = rCalc(f1_kw,f2_kw,f3_kw,f4_kw)
C_m = mCalc(M_costHybrid,M_costHybrid,M_costDiesel,M_costDiesel)
C_f = fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4)
C_tot = totCalc(C_a,C_ab,C_i,C_r,C_m,C_f)

C_opex = opexCalc(C_tot,C_a,C_ab,C_i)
summary(C_a,C_ab,C_i,C_r,C_m,C_f,C_tot,C_opex)


COST SUMMARY [SEK / DAY]

Acquisition cost: 43890.627137192
Acquisition cost (excl. batteries): 42468.30397501744
Battery part of acquisition cost: 1422.3231621745617
Infrastructure cost: 188.80395958069406
Battery replacement cost: 3555.8079054364043
Maintenance cost: 8219.17808219178
Fuel cost: 9174.981
Total scenario cost: 65029.39808440088
Total Opex: 20949.96698762818


In [16]:
#BB

t_op1 = 11     
t_op2 = 10.5
t_op3 = 9
t_op4 = 3

e_f1 = 1
e_f2 = 1
e_f3 = 1
e_f4 = 1

f1_kw = 1260
f2_kw = 1000
f3_kw = 1260
f4_kw = 1260

P = 4
W_s = 0
W_f = 0
P_char = 200
W_char = 0

C_a = aCalc(F_costHybrid,F_costHybrid,F_costHybridRetrofit,F_costHybridRetrofit)
C_ab = abCalc(1260,1000,1260,1260)
C_i = iCalc(P,W_s,W_f,P_char,W_char)
C_r = rCalc(f1_kw,f2_kw,f3_kw,f4_kw)
C_m = mCalc(M_costHybrid,M_costHybrid,M_costHybrid,M_costHybrid)
C_f = fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4)
C_tot = totCalc(C_a,C_ab,C_i,C_r,C_m,C_f)

C_opex = opexCalc(C_tot,C_a,C_ab,C_i)
summary(C_a,C_ab,C_i,C_r,C_m,C_f,C_tot,C_opex)

COST SUMMARY [SEK / DAY]

Acquisition cost: 40519.58488654477
Acquisition cost (excl. batteries): 37511.308463892376
Battery part of acquisition cost: 3008.2764226523923
Infrastructure cost: 377.6079191613881
Battery replacement cost: 7520.69105663098
Maintenance cost: 7671.232876712329
Fuel cost: 7370.0
Total scenario cost: 63459.11673904947
Total Opex: 22561.923933343307


In [17]:
#OBS

t_op1 = 8       
t_op2 = 8.5
t_op3 = 9 
t_op4 = 8

e_f1 = 1
e_f2 = 1
e_f3 = 1
e_f4 = 1

f1_kw = 670
f2_kw = 670
f3_kw = 670
f4_kw = 670

P = 4
W_s = 0
W_f = 0
P_char = 200
W_char = 0

C_a = aCalc(F_costHybrid,F_costHybrid,F_costHybridRetrofit,F_costHybridRetrofit)
C_ab = abCalc(1260,1000,670,670)
C_i = iCalc(P,W_s,W_f,P_char,W_char)
C_r = rCalc(f1_kw,f2_kw,f3_kw,f4_kw)
C_m = mCalc(M_costHybrid,M_costHybrid,M_costHybrid,M_costHybrid)
C_f = fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4)
C_tot = totCalc(C_a,C_ab,C_i,C_r,C_m,C_f)

C_opex = opexCalc(C_tot,C_a,C_ab,C_i)
summary(C_a,C_ab,C_i,C_r,C_m,C_f,C_tot,C_opex)



COST SUMMARY [SEK / DAY]

Acquisition cost: 39776.9559788607
Acquisition cost (excl. batteries): 37511.308463892376
Battery part of acquisition cost: 2265.6475149683283
Infrastructure cost: 377.6079191613881
Battery replacement cost: 4216.621763968833
Maintenance cost: 7671.232876712329
Fuel cost: 7370.0
Total scenario cost: 59412.41853870325
Total Opex: 19257.854640681155


In [18]:
#WS

t_op1 = 8.375       
t_op2 = 8.375  
t_op3 = 8.375  
t_op4 = 8.375  

e_f1 = 1
e_f2 = 1
e_f3 = 1
e_f4 = 1

f1_kw = 130
f2_kw = 130
f3_kw = 130
f4_kw = 130

P = 0
W_s = 2
W_f = 4
P_char = 0
W_char = 700

C_a = aCalc(F_costHybrid,F_costHybrid,F_costHybrid,F_costHybrid)
C_ab = abCalc(130,130,130,130)
C_i = iCalc(P,W_s,W_f,P_char,W_char)
C_r = rCalc(f1_kw,f2_kw,f3_kw,f4_kw)
C_m = mCalc(M_costHybrid,M_costHybrid,M_costHybrid,M_costHybrid)
C_f = fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4)
C_tot = totCalc(C_a,C_ab,C_i,C_r,C_m,C_f)

C_opex = opexCalc(C_tot,C_a,C_ab,C_i)
summary(C_a,C_ab,C_i,C_r,C_m,C_f,C_tot,C_opex)


COST SUMMARY [SEK / DAY]

Acquisition cost: 45929.7099006635
Acquisition cost (excl. batteries): 45602.44970405697
Battery part of acquisition cost: 327.26019660653634
Infrastructure cost: 1387.709102918101
Battery replacement cost: 818.1504915163409
Maintenance cost: 7671.232876712329
Fuel cost: 7370.0
Total scenario cost: 63176.80237181028
Total Opex: 15859.38336822867


In [19]:
#WF

t_op1 = 8.375       
t_op2 = 8.375  
t_op3 = 8.375  
t_op4 = 8.375  

e_f1 = 1
e_f2 = 1
e_f3 = 1
e_f4 = 1

f1_kw = 630
f2_kw = 630
f3_kw = 630
f4_kw = 630

#Special case: 2 stationary 470 kW, 4 stationary 50 kW for overnight charging, 4 on-board 470 kW.

W_s470 = 2
W_s50 = 4
W_f = 4
P_char = 0

C_i = (((W_s470 * 470 * W_cost * 0.6) + (W_s50 * 50 * W_cost * 0.6) + (W_f * 470 * W_cost * 0.4)) * CRF) / 365

C_a = aCalc(F_costHybrid,F_costHybrid,F_costHybrid,F_costHybrid)
C_ab = abCalc(630,630,630,630)
C_r = rCalc(f1_kw,f2_kw,f3_kw,f4_kw)
C_m = mCalc(M_costHybrid,M_costHybrid,M_costHybrid,M_costHybrid)
C_f = fCalc(t_op1,t_op2,t_op3,t_op4,e_f1,e_f2,e_f3,e_f4)
C_tot = totCalc(C_a,C_ab,C_i,C_r,C_m,C_f)

C_opex = opexCalc(C_tot,C_a,C_ab,C_i)
summary(C_a,C_ab,C_i,C_r,C_m,C_f,C_tot,C_opex)


COST SUMMARY [SEK / DAY]

Acquisition cost: 47188.4029645348
Acquisition cost (excl. batteries): 45602.44970405697
Battery part of acquisition cost: 1585.95326047783
Infrastructure cost: 1016.7093223420374
Battery replacement cost: 3964.8831511945746
Maintenance cost: 7671.232876712329
Fuel cost: 7370.0
Total scenario cost: 67211.22831478374
Total Opex: 19006.116027906908
